In [1]:
import pandas as pd
import tushare as ts
import pymysql
import re


In [2]:
ts.set_token('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')
pro = ts.pro_api()
#pro = ts.pro_api('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')




#ts_list = '''000001.SZ,600000.SH,002920.SZ,600089.SH, 600096.SH, 002956.SZ, 600872.SH, 601700.SH,
#        601789.SH, 000049.SZ, 002222.SZ, 300815.SZ, 600653.SH'''
#ts_list = '''000001.SZ,600000.SH,002920.SZ,600089.SH, 600096.SH, 002956.SZ, 600872.SH, 601700.SH'''
#ts_list = '601700.SH'
#df = pro.daily(ts_code=ts_list, start_date='20220616', end_date='20220707')

In [3]:
df = pro.stock_basic()

In [4]:
df

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
4826,871981.BJ,871981,晶赛科技,None,None,北交所,20211115
4827,872925.BJ,872925,锦好医疗,None,None,北交所,20211025
4828,873169.BJ,873169,七丰精工,None,None,北交所,20220415
4829,873223.BJ,873223,荣亿精密,None,None,北交所,20220609


连接数据库

In [5]:
try:
    db = pymysql.connect(host='localhost',
                     user='root',
                     password='Aa123456',
                     database='dbtest',
                     charset='utf8')
    print("数据库连接成功！")
except pymysql.Error as e:
    print("连接数据库错误："+str(e))

数据库连接成功！


创建游标

In [6]:
cur = db.cursor()
sql = "SELECT VERSION()"
cur.execute(sql)
dbf = cur.fetchone()
print(dbf)

('8.0.29',)


创建 Stock_basic 表格，用于存储市场股票基本信息，只运行一次创建，之后只是操作这个表格

In [7]:
def Create_stock_basic_table():
    sql='DROP TABLE IF EXISTS Stock_basic'

    cur.execute(sql)
    db.commit()

    sql = 'CREATE TABLE Stock_basic(id int not null auto_increment PRIMARY  KEY, ts_code CHAR(9) NOT NULL,symbol CHAR(6), name CHAR(16), area CHAR(8),industry CHAR(16),market CHAR(8),list_date CHAR(8) )'
    cur.execute(sql)
    db.commit()

In [8]:
def Stock_basic_insert():
    #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
    #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    sql_insert = "INSERT INTO Stock_basic(ts_code, symbol, name, area, industry, market, list_date) VALUES (%s,%s,%s,%s,%s,%s,%s)"
    for indexs in range(1,df.index.stop):
#indexs =0
        cur.execute(sql_insert,(str(df.loc[indexs,'ts_code']),str(df.loc[indexs,'symbol']),str(df.loc[indexs,'name']),str(df.loc[indexs,'area']),\
                                       str(df.loc[indexs,'industry']),str(df.loc[indexs,'market']),str(df.loc[indexs,'list_date'])))
        db.commit()
        print("写入成功一只股票数据！" + str(indexs))

获取日线行情数据

In [9]:
df = pro.daily(ts_code='000001.SZ', start_date='20220701', end_date='20220712')

In [10]:
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20220712,14.41,14.73,14.38,14.42,14.46,-0.04,-0.2766,742882.42,1078431.990
1,000001.SZ,20220711,14.50,14.57,14.36,14.46,14.54,-0.08,-0.5502,611691.94,883613.754
2,000001.SZ,20220708,14.50,14.65,14.37,14.54,14.42,0.12,0.8322,639698.23,928850.349
3,000001.SZ,20220707,14.54,14.65,14.38,14.42,14.53,-0.11,-0.7571,730160.98,1057696.547
4,000001.SZ,20220706,14.83,14.93,14.46,14.53,14.91,-0.38,-2.5486,1043335.23,1520022.515
5,000001.SZ,20220705,14.99,15.27,14.82,14.91,14.94,-0.03,-0.2008,1115172.10,1674719.025
6,000001.SZ,20220704,14.95,14.98,14.75,14.94,14.92,0.02,0.1340,834968.07,1241453.402
7,000001.SZ,20220701,15.00,15.07,14.84,14.92,14.98,-0.06,-0.4005,779243.32,1164030.776


创建存储日线行情的表， 每个股票创建一个表

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ts_code     8 non-null      object 
 1   trade_date  8 non-null      object 
 2   open        8 non-null      float64
 3   high        8 non-null      float64
 4   low         8 non-null      float64
 5   close       8 non-null      float64
 6   pre_close   8 non-null      float64
 7   change      8 non-null      float64
 8   pct_chg     8 non-null      float64
 9   vol         8 non-null      float64
 10  amount      8 non-null      float64
dtypes: float64(9), object(2)
memory usage: 832.0+ bytes


In [12]:
cur = db.cursor()
sql = "SELECT VERSION()"
cur.execute(sql)
dbf = cur.fetchone()
print(dbf)

('8.0.29',)


In [13]:
def Str_remove_dot(str_org):   
    str = str_org
    a = str.split('.',-1)
    str_new = a[0]+a[1]
    return str_new

创建日线表格，价格字段使用 decimal(8,2)
通过 VALUE 动态改变表名的一种方式
mysql 中字段不能使用带点"." , 不能使用一些关键字， 如 change, modify 等 

In [60]:
def Create_stock_daily_table_2(stock_code):

#    if(len(stock_code) != 9):
#       print("股票代码长度不正确！！")
#       return false
    table_name = '{}{}'.format(stock_code[:6], stock_code[7:])
    print ("Table name:" + table_name)
    
 #   try:
#        VALUE =Str_remove_dot(stock_code)
    sql = """CREATE TABLE {:s} (id int not null auto_increment PRIMARY  KEY, ts_code CHAR(9) NOT NULL,trade_date CHAR(8), open decimal(8,2),
           high decimal(8,2), low decimal(8,2),close decimal(8,2), pre_close decimal(8,2), pri_chg decimal(8,2), pct_chg decimal(10,4), vol decimal(16,2), amount decimal(16,2) )""".format(table_name)
    cur.execute(sql)
    db.commit()
    print ("Create Table: " + table_name)
  #  except:
  #      print("Create Table False!!!")

创建日线表格，价格字段使用字符串

In [49]:
def Create_stock_daily_table(stock_code):

    if(len(stock_code) != 9):
       print("股票代码长度不正确！！")
       return false
    
    try:
        VALUE =Str_remove_dot(stock_code)
        sql = """CREATE TABLE {:s} (id int not null auto_increment PRIMARY  KEY, ts_code CHAR(9) NOT NULL,trade_date CHAR(8), open CHAR(10),
           high CHAR(10), low CHAR(10),close CHAR(10), pre_close CHAR(10), pri_chg CHAR(10), pct_chg CHAR(8), vol CHAR(16), amount CHAR(16) )""".format(VALUE)
        cur.execute(sql)
        db.commit()
        print ("Create Table: " + VALUE)
    except:
        print("Create Table False!!!")

In [50]:
stock_co ='000001.sz'
Create_stock_daily_table(stock_co)

Create Table False!!!


In [51]:
def Del_stock_daily_table(stock_code):
    
    if(len(stock_code) != 9):
       print("股票代码长度不正确！！")
       return false
    
    try:
        VALUE =Str_remove_dot(stock_code)
#        sql = """CREATE TABLE {:s} (id int not null auto_increment PRIMARY  KEY, ts_code CHAR(9) NOT NULL,trade_date CHAR(8), open CHAR(10),
#           high CHAR(10), low CHAR(10),close CHAR(10), pre_close CHAR(10), pri_chg CHAR(10), pct_chg CHAR(8), vol CHAR(16), amount CHAR(16) )""".format(VALUE)
        sql = 'DROP TABLE IF EXISTS {:s}'.format(VALUE)
        cur.execute(sql)
        db.commit()
        print ("Del Table: " + VALUE)
    except:
        print("Del Table Error!!!")

In [52]:
stock_co ='000007.sz'
#Del_stock_daily_table(stock_co)

In [72]:
type(df.loc[2,'high'])

numpy.float64

日线数据写入db

In [54]:
def Insert_stock_daily():
    #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
    #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    sql_insert = """INSERT INTO 000001sz(ts_code, trade_date, open, high, low, close, pre_close, pri_chg, pct_chg, vol,
                  amount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    for indexs in range(0,df.index.stop):
        try:
            cur.execute(sql_insert,(str(df.loc[indexs,'ts_code']),str(df.loc[indexs,'trade_date']),str(df.loc[indexs,'open']),str(df.loc[indexs,'high']),\
                                str(df.loc[indexs,'low']),str(df.loc[indexs,'close']),str(df.loc[indexs,'pre_close']),str(df.loc[indexs,'change']),\
                                str(df.loc[indexs,'pct_chg']),str(df.loc[indexs,'vol']),str(df.loc[indexs,'amount']) ))
            db.commit()
            print("写入成功一条股票日线数据！" + str(indexs))
        except:
            print("写入日线数据失败！！！")

日线写入， 使用 decimal(8,2) 格式

In [81]:
def Insert_stock_daily_2(stock_code):
    #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
    #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    
    table_name = '{}{}'.format(stock_code[:6], stock_code[7:])
    print ("Table name:" + table_name)
    
    sql_insert = """INSERT INTO {:s}(ts_code, trade_date, open, high, low, close, pre_close, pri_chg, pct_chg, vol,
                  amount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""".format(table_name)
    for indexs in range(0,df.index.stop):
        try:
            cur.execute(sql_insert,(str(df.loc[indexs,'ts_code']),str(df.loc[indexs,'trade_date']),str(df.loc[indexs,'open']),str(df.loc[indexs,'high']),\
                                str(df.loc[indexs,'low']),str(df.loc[indexs,'close']),str(df.loc[indexs,'pre_close']),str(df.loc[indexs,'change']),\
                                str(df.loc[indexs,'pct_chg']),str(df.loc[indexs,'vol']),str(df.loc[indexs,'amount']) ))
            db.commit()
            print("写入成功一条股票日线数据！" + str(indexs))
        except:
            print("写入日线数据失败！！！")


In [56]:
Insert_stock_daily()

写入成功一条股票日线数据！0
写入成功一条股票日线数据！1
写入成功一条股票日线数据！2
写入成功一条股票日线数据！3
写入成功一条股票日线数据！4
写入成功一条股票日线数据！5
写入成功一条股票日线数据！6
写入成功一条股票日线数据！7


In [82]:
sc = '000010.sz'
#Create_stock_daily_table_2(sc)
Insert_stock_daily_2(sc)

Table name:000010sz
写入成功一条股票日线数据！0
写入成功一条股票日线数据！1
写入成功一条股票日线数据！2
写入成功一条股票日线数据！3
写入成功一条股票日线数据！4
写入成功一条股票日线数据！5
写入成功一条股票日线数据！6
写入成功一条股票日线数据！7


In [86]:
sql = 'SELECT * FROM 000010sz'
cur.execute(sql)
result = cur.fetchall()
print(result)
print(result[0][3])

((1, '000001.SZ', '20220712', Decimal('14.41'), Decimal('14.73'), Decimal('14.38'), Decimal('14.42'), Decimal('14.46'), Decimal('-0.04'), Decimal('-0.2766'), Decimal('742882.42'), Decimal('1078431.99')), (2, '000001.SZ', '20220711', Decimal('14.50'), Decimal('14.57'), Decimal('14.36'), Decimal('14.46'), Decimal('14.54'), Decimal('-0.08'), Decimal('-0.5502'), Decimal('611691.94'), Decimal('883613.75')), (3, '000001.SZ', '20220708', Decimal('14.50'), Decimal('14.65'), Decimal('14.37'), Decimal('14.54'), Decimal('14.42'), Decimal('0.12'), Decimal('0.8322'), Decimal('639698.23'), Decimal('928850.35')), (4, '000001.SZ', '20220707', Decimal('14.54'), Decimal('14.65'), Decimal('14.38'), Decimal('14.42'), Decimal('14.53'), Decimal('-0.11'), Decimal('-0.7571'), Decimal('730160.98'), Decimal('1057696.55')), (5, '000001.SZ', '20220706', Decimal('14.83'), Decimal('14.93'), Decimal('14.46'), Decimal('14.53'), Decimal('14.91'), Decimal('-0.38'), Decimal('-2.5486'), Decimal('1043335.23'), Decimal('15